In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
!pip install fast-bert

In [ ]:
!git clone https://github.com/NVIDIA/apex
#!cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

import torch
import torch.optim as optim

In [ ]:
from fastai import *
from fastai.vision import *
from fastai.text import *
from fastai.callbacks import *

In [ ]:
from fast_bert.data_cls import BertDataBunch
from sklearn.model_selection import train_test_split



In [ ]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)

config = Config(
    testing=False,
    bert_model_name="bert-base-multilingual-cased",
    max_lr=3e-5,
    epochs=4,
    use_fp16=True,
    bs=4,
    discriminative=False,
    max_seq_len=256,
)

In [ ]:
from pytorch_pretrained_bert import BertTokenizer
bert_tok = BertTokenizer.from_pretrained(
    config.bert_model_name,
)

In [ ]:
OUTPUT_DIR = Path('/kaggle/working/')#@param {type:"string"}

def convert_catogory_to_num(arg):
    switcher = { 
        "art-and-literature": 0,
        "bangladesh": 1,
        "durporobash": 2,
        "economy": 3,
        "education": 4,
        "entertainment": 5,
        "international": 6,
        "life-style": 7,
        "northamerica": 8,
        "opinion": 9,
        "sports": 10,
        "technology": 11,
        }
    return switcher.get(arg, -2) 


In [ ]:
from tensorflow import keras
import os
import re

def convert_each_data(temp):
    data = []
    for i in temp:
        data.append(convert_catogory_to_num(i))
    return data

def load_dataset():
    temp = pd.read_pickle('/kaggle/input/40k_bangla_newspaper_article.p')
    temp = pd.DataFrame(temp)
   # print(temp.head())
   # print(temp["content"][0])
    print(set(temp.category))
    data = {}
    data["content"] = temp["content"]
    data["category"] = convert_each_data(temp["category"])
    #print(data["content"][0])
    return pd.DataFrame.from_dict(data)

def download_and_load_datasets(size):
    data = load_dataset()
    #data = data.drop(data[data["category"] == -2], axis = 1)
    data = data[data.category != -2]
    #print(data.head())
    #print(data[data["category"] == 11])
    #print(set(data["category"]))
    print(data.shape[0])
    
    data.to_csv('/kaggle/working/news.csv',index=False)
    data = pd.read_csv('/kaggle/working/news.csv')
    
    data = data[~data.content.isna()]
    print(data.shape[0])
    
    train, test = train_test_split(data , test_size = size, random_state = 0)
    return train, test

In [ ]:
train,test = download_and_load_datasets(0.2)
train.to_csv('/kaggle/working/train.csv', index = False)
test.to_csv('/kaggle/working/test.csv', index = False)

In [ ]:
DATA_COLUMN = 'content'
LABEL_COLUMN = 'category'
# label_list is the list of labels
label_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
pd.DataFrame(label_list).to_csv('/kaggle/working/label.csv', index = False, header = False)

In [ ]:
testFrame = pd.read_csv('/kaggle/working/train.csv')
testFloat = float(3.0)
count = 0
for i in range(0,testFrame.shape[0]):
    if(type(testFrame['content'][i]) == type(testFloat)):
        print(testFrame['content'][i])
    

In [ ]:
DATA_PATH = Path('/kaggle/working/')
print(DATA_PATH)
LABEL_PATH = DATA_PATH

In [ ]:
print(pd.read_csv('/kaggle/working/label.csv').head(12))
print(pd.read_csv('/kaggle/working/train.csv').head(12))
print(pd.read_csv('/kaggle/working/test.csv').head(12))

In [ ]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer= config.bert_model_name,
                          train_file='train.csv',
                          val_file='test.csv',
                          label_file='label.csv',
                          text_col= DATA_COLUMN,
                          label_col= LABEL_COLUMN,
                          batch_size_per_gpu=16,
                          max_seq_length=256,
                          multi_gpu=True,
                          multi_label= False,
                          model_type='bert')


In [ ]:
databunch.show_batch()

In [ ]:
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy
import logging

logging.basicConfig(level=logging.NOTSET)
logger = logging.getLogger()
device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]

learner = BertLearner.from_pretrained_model(databunch, pretrained_path=config.bert_model_name, metrics=metrics, device=device_cuda,
logger=logger,output_dir=OUTPUT_DIR,finetuned_wgts_path=None,warmup_steps=500,multi_gpu=True,is_fp16=True,multi_label=False,logging_steps=50)

In [ ]:
!pip uninstall -y apex

In [ ]:
!rm -r apex


In [ ]:
!git clone https://github.com/NVIDIA/apex.git

In [ ]:
!cd apex

In [ ]:
!pip install apex

In [ ]:
learner.fit(epochs=5,lr=6e-5,validate=True, schedule_type="warmup_cosine")

In [ ]:
learner.save_model()


In [ ]:
#Finish Finish

In [ ]:
class FastAiBertTokenizer(BaseTokenizer):
    """Wrapper around BertTokenizer to be compatible with fast.ai"""
    def __init__(self, tokenizer: BertTokenizer, max_seq_len: int=128, **kwargs):
        self._pretrained_tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __call__(self, *args, **kwargs):
        return self

    def tokenizer(self, t:str) -> List[str]:
        """Limits the maximum sequence length"""
        return ["[CLS]"] + self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2] + ["[SEP]"]

In [ ]:
fastai_bert_vocab = Vocab(list(bert_tok.vocab.keys()))
fastai_tokenizer = Tokenizer(tok_func=FastAiBertTokenizer(bert_tok, max_seq_len=config.max_seq_len), pre_rules=[], post_rules=[])

In [ ]:
class BertTokenizeProcessor(TokenizeProcessor):
    def __init__(self, tokenizer):
        super().__init__(tokenizer=tokenizer, include_bos=False, include_eos=False)

class BertNumericalizeProcessor(NumericalizeProcessor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, vocab=Vocab(list(bert_tok.vocab.keys())), **kwargs)

def get_bert_processor(tokenizer:Tokenizer=None, vocab:Vocab=None):
    """
    Constructing preprocessors for BERT
    We remove sos/eos tokens since we add that ourselves in the tokenizer.
    We also use a custom vocabulary to match the numericalization with the original BERT model.
    """
    return [BertTokenizeProcessor(tokenizer=tokenizer),
            NumericalizeProcessor(vocab=vocab)]

In [ ]:
class BertDataBunch(TextDataBunch):
    @classmethod
    def from_df(cls, path:PathOrStr, train_df:DataFrame, valid_df:DataFrame, test_df:Optional[DataFrame]=None,
                tokenizer:Tokenizer=None, vocab:Vocab=None, classes:Collection[str]=None, text_cols:IntsOrStrs=1,
                label_cols:IntsOrStrs=0, label_delim:str=None, **kwargs) -> DataBunch:
        "Create a `TextDataBunch` from DataFrames."
        p_kwargs, kwargs = split_kwargs_by_func(kwargs, get_bert_processor)
        # use our custom processors while taking tokenizer and vocab as kwargs
        processor = get_bert_processor(tokenizer=tokenizer, vocab=vocab, **p_kwargs)
        if classes is None and is_listy(label_cols) and len(label_cols) 
        > 1: classes = label_cols
        src = ItemLists(path,TextList.from_df(train_df, path, cols=text_cols, processor=processor),
                        TextList.from_df(valid_df, path, cols=text_cols, processor=processor))
        src = src.label_for_lm() if cls==TextLMDataBunch else src.label_from_df(cols=label_cols, classes=classes)
        if test_df is not None: src.add_test(TextList.from_df(test_df, path, cols=text_cols))
        return src.databunch(**kwargs)

In [ ]:
databunch = BertDataBunch.from_df(".", trainpak, validpak,
                  tokenizer=fastai_tokenizer,
                  vocab=fastai_bert_vocab,
                  text_cols="text",
                  label_cols=label_cols,
                  bs=config.bs,
                  collate_fn=partial(pad_collate, pad_first=False, pad_idx=0),
             )

In [ ]:
databunch.show_batch()

In [ ]:
from pytorch_pretrained_bert.modeling import BertConfig, BertForSequenceClassification
bert_model = BertForSequenceClassification.from_pretrained(config.bert_model_name,num_labels=12)

In [ ]:
def accuracy(input:Tensor, targs:Tensor)->Rank0Tensor:
    "Computes accuracy with `targs` when `input` is bs * n_classes."
    n = targs.shape[0]
    input = input.argmax(dim=-1).view(n,-1)
    targs = targs.view(n,-1).long()
    return (input==targs).float().mean()

In [ ]:
from fastai.callbacks import *

learner = Learner(
    databunch, bert_model,loss_func=nn.BCEWithLogitsLoss(),metrics=accuracy
)
if config.use_fp16: learner = learner.to_fp16()

In [ ]:
lr=6e-3

In [ ]:
learner.fit_one_cycle(7,lr)

In [ ]:
learner.recorder.plot_losses()

In [ ]:
learner.recorder.plot_metrics()